In [1]:
from compute import Var, Par
from datastructures.api import Model, adda, addf, addsolver, setsolvefor, addobj, addeq, addineq, OPT
from datastructures.execution import edges_from_components
from datastructures.graphutils import namefromsympy, dfs_tree, SOLVER, VAR, COMP, nested_sources, sources, edges_to_Ein_Eout, flat_graph_formulation, root_solver
from datastructures.nestedgraph import root_sources
from datastructures.execution import generate_components_and_residuals
from datastructures.workflow import get_f, order_from_tree, default_solver_options, mdao_workflow, implicit_comp_name
from datastructures.workflow_mdao import mdao_workflow_with_args
from datastructures.operators import sort_scc, reorder_merge_solve
from datastructures.assembly import build_archi
from datastructures.api import evalexpr
from datastructures.postprocess import print_inputs, print_outputs, update_varval
import openmdao.api as om
import numpy as np
from sympy import cos

In [2]:
from compute import ureg
ureg.define('USD = [currency] ')

# Variables and parameters 
They are shared accross models

In [ ]:
P_t = Var('P_t', unit='W')
P_t.forceunit=True
Pbus = Var('P_b', unit='W')
Pbus.forceunit=True
S = Var('S', 400, unit='km')
D = Var('D', 0.1, unit='m')
A = Var('A', 0.5, unit='m^2')
eta_t = Par(r'\eta_t', 0.16)
eta_a = Par(r'\eta', 0.5)
alpha = Par(r'\alpha', 1e20, 'bits/W')
th = Par(r'\theta', np.pi/4)
W0 = Par('W_0', 1367, 'W/m^2')
Re = Par('Re', 6378, 'km')
μ = Par('mu', 3.986005e14, 'm^3/s^2')
c1 = Par('c_1', 2500, 'USD/m^2')
c2 = Par('c_2', 12000, 'USD/m^2')
c3 = Par('c_3', 100, 'USD/W')
c4 = Par('c_4', 12000, 'USD')

# Formulation 1

In [3]:
model = Model(solver=OPT)
m = model.root
s2 = addsolver(m)#, solvefor=['Pbus', 'P_t'])
adda(s2, Pbus, 10*P_t**0.5)
adda(s2, P_t, A*eta_a*W0*cos(th)-Pbus)
R = adda(m, 'R', alpha*P_t*D**2*eta_t/(16*S**2), unit='MB')
C = adda(m, 'C', c1*D**2+c2*A+c3*Pbus+c4, unit='kUSD')
addeq(m, -R+1)
addobj(m, C)

In [4]:
setsolvefor(m, [D,A], {D:[0.5,10], A:[0.04,1]})

In [5]:
edges, tree = model.generate_formulation()

In [6]:
ordered_tree = tree
ordered_edges = edges
comp_options = model.comp_options
var_options = model.var_options
solvers_options = model.solvers_options
solvers_options[1].update({'optimizer': 'SLSQP'})
solvers_options[2] = {'solver': 'N', 'maxiter':20}
nodetyperepr = model.nametyperepr
namingfunc = namefromsympy(nodetyperepr)
sequence = order_from_tree(ordered_tree[0], ordered_tree[1], ordered_edges[1])
solvers_options = default_solver_options(ordered_tree, solvers_options)
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)
components = model.components
lookup_f = get_f(components, ordered_edges)
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [13]:
prob, mdao_in, groups = build_archi(ordered_edges, ordered_tree, wfmdao, namingfunc, model.idmapping)

bounds {'upper': 0.0, 'lower': 0.0}


In [14]:
prob.set_val('D', 0.67)
prob.set_val('A', 0.03)

In [15]:
prob.run_driver();


==
s2
==
NL: Newton Converged in 7 iterations

==
s2
==
NL: Newton Converged in 6 iterations

==
s2
==
NL: Newton Converged in 5 iterations

==
s2
==
NL: Newton Converged in 5 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 2 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 2 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 2 iterations

==
s2
==
NL: Newton Converged in 2 iterations

==
s2
==
NL:

In [16]:
mdao_in_dv = mdao_in.union({'D','A'})

In [17]:
print_inputs(model, prob, mdao_in_dv)

,0,1,2
0,S,400,km
1,D,0.67,m
2,A,0.041,m²


In [18]:
print_outputs(model, prob, namingfunc)

,0,1,2
0,P_b,16.874,W
1,P_t,2.847,W
2,R,1,MB
3,C,15.301,kUSD


In [15]:
print(print_inputs(model, mdao_in).style.to_latex())

\begin{tabular}{llr}
 & 0 & 1 \\
0 & A & 0.500000 \\
1 & D & 0.100000 \\
2 & S & 400.000000 \\
\end{tabular}



In [16]:
print(print_outputs(model, prob, namingfunc).style.to_latex())

\begin{tabular}{llll}
 & 0 & 1 & 2 \\
0 & C & 26.063 & kUSD \\
1 & P_b & 80.381 & W \\
2 & P_t & 64.611 & W \\
3 & R & 0.946 & MB \\
\end{tabular}



In [10]:
update_varval(model, prob, namingfunc)

In [11]:
evalexpr(10*P_t**0.5)

dummy

# Restructured

In [19]:
modelrstr = Model(solver=OPT)
mrstr = modelrstr.root
R = Var('R', unit='MB')
adda(mrstr, P_t, R/(alpha*D**2*eta_t)*(16*S**2))
adda(mrstr, Pbus, 10*P_t**0.5)
adda(mrstr, A, (Pbus+P_t)/(eta_a*W0*cos(th)))
C = adda(mrstr, 'C', c1*D**2+c2*A+c3*Pbus+c4, unit='kUSD')
addobj(mrstr, C)

In [20]:
setsolvefor(mrstr, [D], {D:[0,10]})

In [21]:
edges, tree = modelrstr.generate_formulation()

In [22]:
ordered_tree = tree
ordered_edges = edges
comp_options = modelrstr.comp_options
var_options = modelrstr.var_options
solvers_options = modelrstr.solvers_options
#solvers_options[1] = {'solver': 'N', 'maxiter':20}
nodetyperepr = modelrstr.nametyperepr
namingfunc = namefromsympy(nodetyperepr)
sequence = order_from_tree(ordered_tree[0], ordered_tree[1], ordered_edges[1])
solvers_options = default_solver_options(ordered_tree, solvers_options)
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)
components = modelrstr.components
lookup_f = get_f(components, ordered_edges)
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [48]:
prob, mdao_in, groups = build_archi(ordered_edges, ordered_tree, wfmdao, namingfunc, modelrstr.idmapping)

In [49]:
R.varval = 1
prob.set_val('R', 1)
#D.varval = 0.001
prob.set_val('D', 0.43057)

In [50]:
prob.run_model()

In [150]:
prob.run_driver();

Optimization terminated successfully    (Exit mode 0)
            Current function value: [15.30092326]
            Iterations: 5
            Function evaluations: 9
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


In [51]:
mdao_in_dv = mdao_in.union({'D'})

In [52]:
print_inputs(modelrstr, prob, mdao_in_dv)

,0,1,2
0,S,400,km
1,R,1,MB
2,D,0.431,m


In [53]:
print_outputs(modelrstr, prob, namingfunc)

,0,1,2
0,P_t,6.904,W
1,P_b,26.276,W
2,A,0.069,m²
3,C,15.915,kUSD


In [81]:
update_varval(modelrstr, prob, namingfunc)